# 2018.12.10由于很多新建的特征，对于Test并不适用，所以在V3上进行进一步整理

In [217]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import datetime
import gc
gc.collect()

640

In [218]:
import lightgbm as lgb

In [219]:
flowdf=pd.read_csv('../data/flow_train.csv')
trandf=pd.read_csv('../data/transition_train.csv')

In [220]:
start='20180301'
dates=[]
for i in range(1,16):
    dd=datetime.datetime.strptime(start,"%Y%m%d")
    date=dd+datetime.timedelta(days=i)
    dates.append(int(datetime.datetime.strftime(date,"%Y%m%d")))
print(dates)

[20180302, 20180303, 20180304, 20180305, 20180306, 20180307, 20180308, 20180309, 20180310, 20180311, 20180312, 20180313, 20180314, 20180315, 20180316]


In [221]:
trandf['start_address']=trandf['o_city_code'].astype(str)+':'+trandf['o_district_code'].astype(str)
trandf['end_address']=trandf['d_city_code'].astype(str)+':'+trandf['d_district_code'].astype(str)

flowdf['address']=flowdf['city_code'].astype(str)+':'+flowdf['district_code'].astype(str)

In [222]:
start_address=list(set(trandf['start_address']))
end_address=list(set(trandf['end_address']))
# print(len(set(start_address)&set(end_address)))
# print(len(start_address))
# print(len(end_address))

### ==========没办法，得先建立个testdf，否则脑子转不动====

### / 代表了除号  
### % 代表了取余

In [223]:
def get_df(k=98,add_list=start_address):
    df=pd.DataFrame()
    df['row_id']=np.arange(9604)
    df['start_address']='0'
    df['end_address']='0'
    for i in range(0,9603,98):
        df['start_address'][i:(i+k)]=add_list[int(i/k)]
        for index in range(i,i+k):
            df.loc[index,'end_address']=add_list[int(index%k)]
    return df

test=get_df(k=98,add_list=start_address)
test['date_dt']=dates[0]

for i in range(1,15,1):
    df=get_df(k=98,add_list=start_address)
    df['date_dt']=dates[i]
    test=pd.concat([test,df],axis=0)
    del df
    gc.collect()

test=test[test['start_address']!=test['end_address']]
print(test.shape)

del test['row_id']

test['o_city_code']=test['start_address'].apply(lambda x:str(x).split(':')[0])
test['o_district_code']=test['start_address'].apply(lambda x:str(x).split(':')[1])
test['d_city_code']=test['end_address'].apply(lambda x:str(x).split(':')[0])
test['d_district_code']=test['end_address'].apply(lambda x:str(x).split(':')[1])

(142590, 4)


# ==========哦哟，以上终于建立好了Testdf========

In [224]:
from datetime import datetime

In [225]:
def month(date):
    date=str(date)
    return int(date[4:6])

def day(date):
    date=str(date)
    return int(date[6:8])

def weekd(date):
    date=str(date)
    return str(datetime.strptime(date,'%Y%m%d').weekday())

In [226]:
gc.collect()

250

In [227]:
trandf['month']=trandf['date_dt'].apply(month)
trandf['month']=trandf['month'].apply(lambda x:int(x+12) if x <4 else x)
trandf['day']=trandf['date_dt'].apply(day)
trandf['weekd']=trandf['date_dt'].apply(weekd)

test['month']=test['date_dt'].apply(month)
test['month']=test['month'].apply(lambda x:int(x+12) if x<4 else x)
test['day']=test['date_dt'].apply(day)
test['weekd']=test['date_dt'].apply(weekd)
test['cnt']=0

flowdf['month']=flowdf['date_dt'].apply(month)
flowdf['month']=flowdf['month'].apply(lambda x:int(x+12) if x <4 else x)
flowdf['day']=flowdf['date_dt'].apply(day)
flowdf['weekd']=flowdf['date_dt'].apply(weekd)

## 对于这个address,流进与流出谁大呢？可以用于flowdf中的特征！！！

# ========数据合并后一起处理=======

In [228]:
data=pd.concat([trandf,test],axis=0)
data['cnt']=round(data['cnt'],6)

In [229]:
#标签数值化
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

le.fit(data['o_city_code'])
data['o_city_code']=le.transform(data['o_city_code'])
data['d_city_code']=le.transform(data['d_city_code'])

le.fit(data['o_district_code'])
data['o_district_code']=le.transform(data['o_district_code'])
data['d_district_code']=le.transform(data['d_district_code'])

le.fit(data['start_address'])
data['start_address']=le.transform(data['start_address'])
flowdf['address']=le.transform(flowdf['address'])
data['end_address']=le.transform(data['end_address'])

In [230]:
#数值onehotcoding,维度太高机器跑不动
# from sklearn.preprocessing import OneHotEncoder
# hot=OneHotEncoder()
# hot_col=['month','day','weekd','o_city_code','o_district_code','d_city_code','d_district_code']
# for fea in hot_col:
#     data[fea+'_hot']=hot.fit_transform(data[[fea]])

# <font color=red>基本特征准备<font>

### <font color=blue>mean  ·  nunique ·  count特征<font>

In [231]:
timefea=['day','weekd']
addfea=['start_address','end_address']

#时间一维
for tf in timefea:
    temp=data[[tf,'cnt']]
    meandf=temp.groupby(tf).cnt.mean().reset_index(name=tf+'_cnt_mean')
    nundf=temp.groupby('cnt')[tf].nunique().reset_index(name=tf+'_nun')
    cntdf=temp.groupby('cnt')[tf].count().reset_index(name='cnt_'+tf+'_count')
    cntdf1=temp.groupby(tf).cnt.count().reset_index(name=tf+'_cnt_count')
    
    data=data.merge(meandf,on=tf,how='left')
    data=data.merge(nundf,on='cnt',how='left')
    data=data.merge(cntdf,on='cnt',how='left')
    data=data.merge(cntdf1,on=tf,how='left')
print(data.shape)
gc.collect()

(2622910, 19)


409

In [232]:
#地址一维
for af in addfea:
    temp=data[[af,'cnt']]
    meandf=temp.groupby(af).cnt.mean().reset_index(name=af+'_cnt_mean')
    nundf=temp.groupby('cnt')[af].nunique().reset_index(name=af+'_nun')
    cntdf=temp.groupby(af).cnt.count().reset_index(name=af+'_cnt_count')
    cntdf1=temp.groupby('cnt')[af].count().reset_index(name='cnt_'+af+'_count')

    data=data.merge(meandf,on=af,how='left')
    data=data.merge(cntdf,on=af,how='left')
    data=data.merge(nundf,on='cnt',how='left')
    data=data.merge(cntdf1,on='cnt',how='left')
print(data.shape)
gc.collect()

(2622910, 27)


288

In [233]:
#时间二维
temp=data[['cnt','day','weekd']]
meandf=temp.groupby(['day','weekd']).cnt.mean().reset_index(name='day_week_cnt_mean')
cntdf1=temp.groupby(['day','weekd']).cnt.count().reset_index(name='day_week__cnt_count')
cntdf2=temp.groupby(['cnt','day']).cnt.count().reset_index(name='cnt_day_count')
# cntdf3=tmp.groupby(['cnt','day']).day.count().reset_index(name='cnt_day_day_count') #与上面的相等
cntdf3=temp.groupby(['cnt','weekd']).cnt.count().reset_index(name='cnt_week_count')
cntdf4=temp.groupby(['cnt','day','weekd']).cnt.count().reset_index(name='cnt_day_week_count')

data=data.merge(meandf,on=['day','weekd'],how='left')
data=data.merge(cntdf1,on=['day','weekd'],how='left')
data=data.merge(cntdf2,on=['cnt','day'],how='left')
data=data.merge(cntdf3,on=['cnt','weekd'],how='left')
data=data.merge(cntdf4,on=['cnt','day','weekd'],how='left')
print(data.shape)

(2622910, 32)


In [234]:
#地址二维
temp=data[['cnt','start_address','end_address']]
meandf=temp.groupby(['start_address','end_address']).cnt.mean().reset_index(name='start_end_cnt_mean')
cntdf=temp.groupby(['start_address','end_address']).cnt.count().reset_index(name='start_end_cnt_count')
# cntdf1=temp.groupby(['cnt','start_address']).cnt.count().reset_index(name='cnt_str_count')
# cntdf2=temp.groupby(['cnt','end_address']).cnt.count().reset_index(name='cnt_end_count')
cntdf3=temp.groupby(['cnt','start_address','end_address']).cnt.count().reset_index(name='cnt_str_end_count')

data=data.merge(meandf,on=['start_address','end_address'],how='left')
data=data.merge(cntdf,on=['start_address','end_address'],how='left')
data=data.merge(cntdf3,on=['cnt','start_address','end_address'],how='left')
print(data.shape)
gc.collect()

(2622910, 35)


301

In [235]:
# #时间地址交叉维度
for tf in timefea:
    for af in addfea:
        temp=data[[tf,af,'cnt']]
        meandf=temp.groupby([tf,af]).cnt.mean().reset_index(name=tf+'_'+af+'_cnt_mean')
        cntdf=temp.groupby([tf,af]).cnt.count().reset_index(name=tf+'_'+af+'_cnt_count')
        cntdf1=temp.groupby(['cnt',tf,af]).cnt.count().reset_index(name='cnt_'+tf+af+'_count')
        data=data.merge(meandf,on=[tf,af],how='left')
        data=data.merge(cntdf,on=[tf,af],how='left')
        data=data.merge(cntdf1,on=['cnt',tf,af],how='left')
print(data.shape)
gc.collect()

(2622910, 47)


154

In [236]:
#时间二维与单地址的交叉
for af in addfea:
    temp=data[['day','weekd',af,'cnt']]
    meandf=temp.groupby(['day','weekd',af]).cnt.mean().reset_index(name='day_week_'+af+'_cnt_mean')
    cntdf=temp.groupby(['day','weekd',af]).cnt.count().reset_index(name='day_week_'+af+'_cnt_count')
    data=data.merge(meandf,on=['day','weekd',af],how='left')
    data=data.merge(cntdf,on=['day','weekd',af],how='left')
print(data.shape)

(2622910, 51)


In [237]:
#时间二维与地址二维度的交叉
temp=data[['day','weekd','start_address','end_address','cnt']]
meandf=temp.groupby(['day','weekd','start_address','end_address']).cnt.mean().reset_index(name='day_week_st_end_cnt_mean')
cntdf=temp.groupby(['day','weekd','start_address','end_address']).cnt.count().reset_index(name='day_week_st_end_cnt_count')
data=data.merge(meandf,on=['day','weekd','start_address','end_address'],how='left')
data=data.merge(cntdf,on=['day','weekd','start_address','end_address'],how='left')
print(data.shape)
del temp
del meandf
del nundf
del cntdf
del cntdf1
del cntdf2
del cntdf3
del cntdf4
gc.collect()

(2622910, 53)


230

### 历史转移率特征，来挖掘历史转移信息，同时防止过拟合（参考历史点击率特征）

In [238]:
data['period'] = data['month']
for feat_1 in addfea:
    res = pd.DataFrame()
    temp = data[[feat_1, 'period', 'cnt']]
    for period in range(6, 16):
        if period == 6:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=feat_1 + '_1')
        else:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=feat_1 + '_1')
        count[feat_1 + '_1'] = count1[feat_1 + '_1']
        count.fillna(value=0, inplace=True)
        count[feat_1 + '_rate'] = round(count[feat_1 + '_1'] / count[feat_1 + '_all'], 6)
        count['period'] = period
        count.drop([feat_1 + '_all', feat_1 + '_1'], axis=1, inplace=True)
        count.fillna(value=0, inplace=True)
        res = res.append(count, ignore_index=True)
    print(feat_1, ' over')
    data = pd.merge(data, res, how='left', on=[feat_1, 'period'])
print(data.shape)
gc.collect()

start_address  over
end_address  over
(2622910, 56)


422

In [239]:
data['period'] = data['day']
for feat_1 in addfea:
    res = pd.DataFrame()
    temp = data[[feat_1, 'period', 'cnt']]
    for period in range(1, 32):
        if period == 1:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=feat_1 + '_1')
        else:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=feat_1 + '_1')
        count[feat_1 + '_1'] = count1[feat_1 + '_1']
        count.fillna(value=0, inplace=True)
        count[feat_1 + '_rate'] = round(count[feat_1 + '_1'] / count[feat_1 + '_all'], 6)
        count['period'] = period
        count.drop([feat_1 + '_all', feat_1 + '_1'], axis=1, inplace=True)
        count.fillna(value=0, inplace=True)
        res = res.append(count, ignore_index=True)
    print(feat_1, ' over')
    data = pd.merge(data, res, how='left', on=[feat_1, 'period'])
print(data.shape)
gc.collect()

start_address  over
end_address  over
(2622910, 58)


1205

In [241]:
data['period'] = data['weekd'].astype(int)
for feat_1 in addfea:
    res = pd.DataFrame()
    temp = data[[feat_1, 'period', 'cnt']]
    for period in range(0, 7):
        if period == 0:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] <= period).values].sum()).reset_index(name=feat_1 + '_1')
        else:
            count = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].count()).reset_index(name=feat_1 + '_all')
            count1 = temp.groupby([feat_1]).apply(
                lambda x: x['cnt'][(x['period'] < period).values].sum()).reset_index(name=feat_1 + '_1')
        count[feat_1 + '_1'] = count1[feat_1 + '_1']
        count.fillna(value=0, inplace=True)
        count[feat_1 + '_rate'] = round(count[feat_1 + '_1'] / count[feat_1 + '_all'], 6)
        count['period'] = period
        count.drop([feat_1 + '_all', feat_1 + '_1'], axis=1, inplace=True)
        count.fillna(value=0, inplace=True)
        res = res.append(count, ignore_index=True)
    print(feat_1, ' over')
    data = pd.merge(data, res, how='left', on=[feat_1, 'period'])
print(data.shape)
gc.collect()

start_address  over
end_address  over
(2622910, 60)


641

### 不行了，随便挖掘几个Ration特征

In [242]:
# ration:day/weekd偏好的ration比例特征（如1号从1地转移的cnt sum占1号所有转移的cnt sum比例）
# 1号那天的Cnt sum/all day cnt sum,该特征舍弃

# 1号那天,1地出发的cnt sum/1号那天all地出发的cnt sum
# on='day','start_address'

# 1号那天,1地出发,到达2地的cnt sum/1号那天，1地出发，到达all地的cnt sum
# on='day','start_address','end_address'


# 0weekd那天的Cnt sum/all weekd cnt sum

# 0weekd那天,1地出发的cnt sum/0weekd那天all地出发的cnt sum

# 0weekd那天,1地出发,到达2地的cnt sum/0weekd那天，1地出发，到达all地的cnt sum


# temp=data[['cnt','day']].groupby('day').cnt.sum().reset_index(name='day_sum')
# data=merge(temp,on='day',how='left')
# data['day_ration']=data['cnt']/

## 来一波像模像样的特征选择吧

In [243]:
# 方差检验
# np.var()

In [244]:
# 2. 单变量特征选择 (Univariate feature selection)
# 　　单变量特征选择的原理是分别单独的计算每个变量的某个统计指标，根据该指标来判断哪些指标重要，
#     剔除那些不重要的指标。
# 　　对于分类问题(y离散)，可采用：
# 　　　　卡方检验，f_classif, mutual_info_classif，互信息
# 　　对于回归问题(y连续)，可采用：
# 　　　　皮尔森相关系数，f_regression, mutual_info_regression，最大信息系数
    
#     这种方法比较简单，易于运行，易于理解，通常对于理解数据有较好的效果（但对特征优化、提高泛化能力来说不一定有效）。
#     这种方法有许多改进的版本、变种。
# 　　单变量特征选择基于单变量的统计测试来选择最佳特征。
# 它可以看作预测模型的一项预处理。
# ==Scikit-learn将特征选择程序用包含 transform 函数的对象来展现==：
# •	SelectKBest 移除得分前 k 名以外的所有特征(取top k)
# •	SelectPercentile 移除得分在用户指定百分比以后的特征(取top k%)

# 将特征输入到评分函数，返回一个单变量的f_score(F检验的值)或p-values(P值，假设检验中的一个标准，P-value用来和显著性水平作比较)，注意SelectKBest 和 SelectPercentile只有得分，没有p-value。
# •	For classification: chi2, f_classif, mutual_info_classif
# •	For regression: f_regression, mutual_info_regression

In [245]:
#特征选择参考
# user_tags特征我们分别用了卡方检验和特征重要性。
# train_new = pd.DataFrame()
# test_new = pd.DataFrame()
# cntv = CountVectorizer()
# cntv.fit(data['user_tags'])
# train_a = cntv.transform(train['user_tags'])
# test_a = cntv.transform(test['user_tags'])
# train_new = sparse.hstack((train_new, train_a), 'csr', 'bool')
# test_new = sparse.hstack((test_new, test_a), 'csr', 'bool')
# # 卡方检验
# SKB = SelectPercentile(chi2, percentile=95).fit(train_new, train_y)
# train_new = SKB.transform(train_new)
# test_new = SKB.transform(test_new)


# size = 300
# x = np.random.normal(0, 1, size)
# # pearsonr(x, y)的输入为特征矩阵和目标向量
# print("Lower noise", pearsonr(x,y))
# print("Higher noise", pearsonr(x, y))
# 这个例子中，我们比较了变量在加入噪音之前和之后的差异。当噪音比较小的时候，相关性很强，p-value很低。

In [247]:
# from sklearn.feature_selection import SelectKBest
# from scipy.stats import pearsonr
# train_new=
# train_y=
# test_new=
# SKB = SelectPercentile(f_regression, percentile=80).fit(train_new, train_y)
# train_new = SKB.transform(train_new)
# test_new = SKB.transform(test_new)

In [248]:
# 模型重要性，或者 三个模型的重要性排序后取交集 或并集

In [249]:
# 蛇佬的贪心算法+退火算法

# =========准备模型========

In [250]:
#实数序列数据预测问题，可以使用平方根误差
# def rmse(y_true,y_pred):
#     rmse=round(np.sqrt(np.average((y_pred-np.array(y_true))**2)),5)
#     return rmse


#这个评估标准是大赛规定的
def rmsle(y_true, y_pred):
    return 'RMSLE', round(np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))),5), False

# from sklearn.metrics import mean_squared_error
#也可以使用mse评价标准

In [251]:
# ======按照出发城市进行分集存储后提取训练==============

#按照出发城市进行分集存储
def save_data(path,data):
    for i in range(0,7):
        ndata=data[data['o_city_code']==i]
        ndata.to_csv(path+'data_'+str(i)+'.csv',index=False)
        
path='../data/'
save_data(path,data)

gc.collect()

176

In [261]:
gc.collect()

245

In [266]:
# def count_corr(df):
#     '''
#     输入dataframe
#     输出相关系数dataframe:col_1,col_2,cor(不包含同一特征且已去重复)
#     '''
#     x = df.corr().abs().unstack().sort_values(ascending=False).reset_index()
#     x = x.loc[x.level_0!=x.level_1]
#     x2 = pd.DataFrame([sorted(i) for i in x[['level_0','level_1']].values])
#     x2['cor'] = x[0].values
#     x2.columns = ['col_1','col_2','cor']
#     return x2.drop_duplicates()

# testn=count_corr(test)

# print(testn)

In [19]:
from sklearn.model_selection import train_test_split

In [ ]:
#由于2018年3月，仅仅出现了1天的已知数据，其他15天均需要预测的，
# ，故将3月亦作为test1数据，然后将test1_true,与test1_pred作对比，3月为年后交通高峰，所以可以普遍加一个定数cnt
# ，或者根据start_address来进行加一定的数后，便是最后的3.2-3.16的预测cnt了

In [35]:
# class lightgbm.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, 
#                              n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, 
#                              min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
#                              subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
#                              random_state=None, n_jobs=-1, silent=True, importance_type='split', **kwargs)

def get_lgb(path,i,N):
    data=pd.read_csv(path+'data_'+str(i)+'.csv')
    train=data[data['cnt']!=0][data['month']!=15]
    test=data[data['cnt']==0]
    test1=data[data['cnt']!=0][data['month']==15]
    test1['month']=11
    print(train.shape,test.shape,test1.shape)
    
    train_xy,off_xy=train_test_split(train,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','date_dt']
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=test.drop(drop_list,axis=1)
    test_x1=test1.drop(drop_list,axis=1)

    gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=50,
                        learning_rate=0.05,
                        n_estimators=N)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='l1',
        early_stopping_rounds=50)

    y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
    y_eval=rmsle(y_val,y_pred)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
    print('The rmlse of prediction is:',y_eval)
    # print('Feature importances:', list(gbm.feature_importances_))

    y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
    y_off_eval=rmsle(y_off, y_pred_off)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_off, y_pred_off) ** 0.5)
    print('The rmlse of prediction is:',y_off_eval)

    gc.collect()
    
    test['cnt']=gbm.predict(test_x,num_iteration=gbm.best_iteration_)
    
    data=pd.concat([train,test],axis=0)
    data.to_csv(path+'dataf_'+str(i)+'.csv',index=False)
    
    return y_eval,y_off_eval

In [37]:
path='../data/'
rmsle0=get_lgb(path,0,N=942)

(569813, 11) (32010, 11)
[1]	training's l1: 2.0359	valid_1's l1: 2.05213
Training until validation scores don't improve for 5 rounds.
[2]	training's l1: 1.97385	valid_1's l1: 1.98925
[3]	training's l1: 1.9142	valid_1's l1: 1.92863
[4]	training's l1: 1.85798	valid_1's l1: 1.87192
[5]	training's l1: 1.80587	valid_1's l1: 1.81928
[6]	training's l1: 1.7558	valid_1's l1: 1.76869
[7]	training's l1: 1.70887	valid_1's l1: 1.72125
[8]	training's l1: 1.66306	valid_1's l1: 1.67495
[9]	training's l1: 1.61885	valid_1's l1: 1.63
[10]	training's l1: 1.57942	valid_1's l1: 1.59038
[11]	training's l1: 1.54075	valid_1's l1: 1.55161
[12]	training's l1: 1.50447	valid_1's l1: 1.51515
[13]	training's l1: 1.47095	valid_1's l1: 1.4817
[14]	training's l1: 1.43872	valid_1's l1: 1.44934
[15]	training's l1: 1.40648	valid_1's l1: 1.41702
[16]	training's l1: 1.37834	valid_1's l1: 1.38887
[17]	training's l1: 1.34956	valid_1's l1: 1.36002
[18]	training's l1: 1.32299	valid_1's l1: 1.33336
[19]	training's l1: 1.29764	va

[161]	training's l1: 0.59263	valid_1's l1: 0.603803
[162]	training's l1: 0.59134	valid_1's l1: 0.602513
[163]	training's l1: 0.588677	valid_1's l1: 0.599874
[164]	training's l1: 0.587204	valid_1's l1: 0.598245
[165]	training's l1: 0.585392	valid_1's l1: 0.596426
[166]	training's l1: 0.584271	valid_1's l1: 0.595374
[167]	training's l1: 0.583593	valid_1's l1: 0.594759
[168]	training's l1: 0.58206	valid_1's l1: 0.593132
[169]	training's l1: 0.580569	valid_1's l1: 0.591608
[170]	training's l1: 0.579766	valid_1's l1: 0.590829
[171]	training's l1: 0.57838	valid_1's l1: 0.589402
[172]	training's l1: 0.575989	valid_1's l1: 0.587025
[173]	training's l1: 0.575087	valid_1's l1: 0.586219
[174]	training's l1: 0.574767	valid_1's l1: 0.585936
[175]	training's l1: 0.573401	valid_1's l1: 0.584516
[176]	training's l1: 0.571207	valid_1's l1: 0.58234
[177]	training's l1: 0.570124	valid_1's l1: 0.581246
[178]	training's l1: 0.567182	valid_1's l1: 0.578348
[179]	training's l1: 0.565041	valid_1's l1: 0.57622

[317]	training's l1: 0.42924	valid_1's l1: 0.443001
[318]	training's l1: 0.427545	valid_1's l1: 0.441381
[319]	training's l1: 0.427259	valid_1's l1: 0.441119
[320]	training's l1: 0.427113	valid_1's l1: 0.441023
[321]	training's l1: 0.426258	valid_1's l1: 0.440142
[322]	training's l1: 0.425451	valid_1's l1: 0.439329
[323]	training's l1: 0.424391	valid_1's l1: 0.438227
[324]	training's l1: 0.423928	valid_1's l1: 0.437755
[325]	training's l1: 0.423249	valid_1's l1: 0.437075
[326]	training's l1: 0.423018	valid_1's l1: 0.436848
[327]	training's l1: 0.422979	valid_1's l1: 0.436793
[328]	training's l1: 0.422741	valid_1's l1: 0.43657
[329]	training's l1: 0.421053	valid_1's l1: 0.434851
[330]	training's l1: 0.420685	valid_1's l1: 0.434447
[331]	training's l1: 0.420434	valid_1's l1: 0.434182
[332]	training's l1: 0.41952	valid_1's l1: 0.433234
[333]	training's l1: 0.418744	valid_1's l1: 0.432443
[334]	training's l1: 0.418547	valid_1's l1: 0.432276
[335]	training's l1: 0.418504	valid_1's l1: 0.432

[475]	training's l1: 0.364553	valid_1's l1: 0.379937
[476]	training's l1: 0.364027	valid_1's l1: 0.379427
[477]	training's l1: 0.363936	valid_1's l1: 0.37935
[478]	training's l1: 0.362974	valid_1's l1: 0.378393
[479]	training's l1: 0.362807	valid_1's l1: 0.378256
[480]	training's l1: 0.362704	valid_1's l1: 0.378161
[481]	training's l1: 0.362549	valid_1's l1: 0.378054
[482]	training's l1: 0.362291	valid_1's l1: 0.37781
[483]	training's l1: 0.362225	valid_1's l1: 0.377748
[484]	training's l1: 0.362122	valid_1's l1: 0.377672
[485]	training's l1: 0.362044	valid_1's l1: 0.377617
[486]	training's l1: 0.362005	valid_1's l1: 0.377596
[487]	training's l1: 0.361897	valid_1's l1: 0.377497
[488]	training's l1: 0.361739	valid_1's l1: 0.377371
[489]	training's l1: 0.360766	valid_1's l1: 0.376396
[490]	training's l1: 0.36059	valid_1's l1: 0.376259
[491]	training's l1: 0.360447	valid_1's l1: 0.376157
[492]	training's l1: 0.360189	valid_1's l1: 0.375942
[493]	training's l1: 0.359243	valid_1's l1: 0.375

[635]	training's l1: 0.324774	valid_1's l1: 0.342823
[636]	training's l1: 0.324636	valid_1's l1: 0.342701
[637]	training's l1: 0.324567	valid_1's l1: 0.342608
[638]	training's l1: 0.324144	valid_1's l1: 0.342206
[639]	training's l1: 0.324116	valid_1's l1: 0.342165
[640]	training's l1: 0.323799	valid_1's l1: 0.341866
[641]	training's l1: 0.32345	valid_1's l1: 0.341487
[642]	training's l1: 0.323021	valid_1's l1: 0.34107
[643]	training's l1: 0.322831	valid_1's l1: 0.340875
[644]	training's l1: 0.32238	valid_1's l1: 0.340432
[645]	training's l1: 0.322285	valid_1's l1: 0.340361
[646]	training's l1: 0.321889	valid_1's l1: 0.339958
[647]	training's l1: 0.321756	valid_1's l1: 0.339833
[648]	training's l1: 0.321241	valid_1's l1: 0.339359
[649]	training's l1: 0.321148	valid_1's l1: 0.339304
[650]	training's l1: 0.320788	valid_1's l1: 0.338908
[651]	training's l1: 0.32069	valid_1's l1: 0.338821
[652]	training's l1: 0.320205	valid_1's l1: 0.338384
[653]	training's l1: 0.320087	valid_1's l1: 0.3383

[791]	training's l1: 0.294269	valid_1's l1: 0.314298
[792]	training's l1: 0.294114	valid_1's l1: 0.314152
[793]	training's l1: 0.294043	valid_1's l1: 0.314084
[794]	training's l1: 0.294011	valid_1's l1: 0.314062
[795]	training's l1: 0.293885	valid_1's l1: 0.31394
[796]	training's l1: 0.293864	valid_1's l1: 0.313948
[797]	training's l1: 0.293792	valid_1's l1: 0.313959
[798]	training's l1: 0.293567	valid_1's l1: 0.313746
[799]	training's l1: 0.29314	valid_1's l1: 0.313359
[800]	training's l1: 0.2931	valid_1's l1: 0.313341
[801]	training's l1: 0.293074	valid_1's l1: 0.313317
[802]	training's l1: 0.292907	valid_1's l1: 0.313157
[803]	training's l1: 0.292716	valid_1's l1: 0.312984
[804]	training's l1: 0.292596	valid_1's l1: 0.312869
[805]	training's l1: 0.292421	valid_1's l1: 0.312699
[806]	training's l1: 0.292357	valid_1's l1: 0.312653
[807]	training's l1: 0.292164	valid_1's l1: 0.312447
[808]	training's l1: 0.292139	valid_1's l1: 0.31243
[809]	training's l1: 0.291804	valid_1's l1: 0.31207

KeyboardInterrupt: 

In [ ]:
rmsle2=get_lgb(path,2,N=935)

In [ ]:
rmsle1=get_lgb(path,1,N=1500)


In [ ]:
rmsle5=get_lgb(path,5,N=1500)

In [ ]:
rmsle3=get_lgb(path,3,N=700)
rmsle4=get_lgb(path,4,N=700)

In [ ]:
rmsle6=get_lgb(path,6,N=300) 